# Lab 6: Regularization and Optimization

Ren Yi, 3-28-2019

The goal of this lab is to learn how to apply different regularization and optimization strategies in PyTorch using MNIST data.

Here a imcomplete list of the techniques we've covered in class
- Regularization
    - L1/L2 regularization
    - Data augmentation
    - Dropout
    - Batch normalization
    - Early stopping
- Optimization
    - SGD (with momentum)
    - Nesterov momentum
    - AdaGrad
    - RMSProp
    - Adam
    
We will show you how some of these methods are used in PyTorch. 

## Problem Setup

In [293]:
import pickle 
import time
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [315]:
# Initialize necessary parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(717)

seed = 345
batch_size = 50
test_batch_size = 50
input_size = 28 * 28
output_size = 10
n_feature = 3

In [295]:
# Data Loader
trainset = datasets.MNIST('data', train=True, download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]))
testset = datasets.MNIST('data', train=False, 
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))]))

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)

In [310]:
## show some images
plt.figure()
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    plt.imshow(image.squeeze().numpy())

In [297]:
def train(model, optimizer, num_epoch=1, print_every=100):
    train_losses =[]
    test_losses =[]
    start_time = time.time()

    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(num_epoch):
        # Training step
        model.train()
        train_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % print_every == 0:
                print('Train set | epoch: {:3d} | {:6d}/{:6d} batches | Loss: {:6.4f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset), loss.item()))
                train_loss += loss.item()

                train_loss /=len(train_loader)
                train_losses.append(train_loss)

        # testing step
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += loss_fn(output, target).item()
                pred = output.data.max(1)[1] # get the index of the max log-probability
                correct += pred.eq(target.data).cpu().sum()

            test_loss /= len(test_loader) # loss function already averages over batch size
            elapse = time.strftime('%H:%M:%S', time.gmtime(int((time.time() - start_time))))
            print('Test set | Average loss: {:6.4f} | Accuracy: {:4.2f}% | time elapse: {:>9}'.format(
                test_loss, 100. * correct / len(test_loader.dataset), elapse))

            test_losses.append(test_loss)

    return train_losses, test_losses

def populate_result(dictionary, method, train_loss, val_loss):
    dictionary[method] = {}
    dictionary[method]['train_loss'] = np.array(train_loss)
    dictionary[method]['val_loss'] = np.array(val_loss)
    
def plot_loss(result, loss='train_loss', ylim=None):
    plt.plot(result['Baseline'][loss], label='Baseline')
    for k in result.keys():
        if k != 'Baseline':
            plt.plot(result[k][loss], label=k)
    if ylim is not None:
        plt.ylim(ylim)
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_best_loss(result, loss='val_loss'):
    labels = ['Baseline']
    acc = [np.max(result['Baseline'][loss])]
    for k in result.keys():
        if k != 'Baseline':
            labels.append(k)
            acc.append(np.max(result[k][loss]))

    x = np.arange(len(labels))
    plt.barh(x, acc)
    plt.yticks(x, labels)
    plt.xlabel('Loss')
    plt.show()

### Baseline method

We will later show how different optimization and regularization techniques can improve baseline model performance. But first,
1. What's our baseline model architecture?
2. What's the optimization method used to train the baseline model?
3. How does this optimization method update its parameters.
$$\theta_{t+1} = \theta_{t} - \eta \nabla J(\theta_{t})$$
where $\eta$ denotes the learning rate

In [298]:
class Baseline(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(Baseline, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Sequential(nn.Conv2d(1, n_feature, kernel_size=5),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(nn.Conv2d(n_feature, n_feature, kernel_size=5),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
        self.fc = nn.Sequential(nn.Linear(n_feature*4*4, 50),
                                nn.ReLU(),
                                nn.Linear(50, 10))


    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc(x)
        return x

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
optimizer = optim.SGD(model.parameters(),lr=0.01)
train_losses, test_losses = train(model, optimizer)
populate_result(optim_results, 'Baseline', train_losses, test_losses)

## Fancier Optimization

https://pytorch.org/docs/stable/optim.html

### SGD with momentum
1. How does SGD with momentum update its parameters?
$$v_{t+1} = \rho v_{t} + \nabla J(\theta_{t})$$
$$\theta_{t+1} = \theta_{t} - \eta v_{t+1}$$
where $v$ and $\rho$ denote velocity and momentum, respectively.
2. Check out the documentation for SGD in PyTorch and complete the code below (Set momentum=0.5). 

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
############ Your code below ############

############       End       ############
train_losses, test_losses = train(model, optimizer)
populate_result(optim_results, 'SGD_momentum', train_losses, test_losses)

### Nesterov momentum

Make a minor change in the above code to apply Nesterov momentum (Set momentum=0.5). 

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
############ Your code below ############

############       End       ############
train_losses, test_losses = train(model, optimizer)
populate_result(optim_results, 'Nesterov_momentum', train_losses, test_losses)

### AdaGrad

1. What's the motivation of AdaGrad?
2. Check out the documentation of AdaGrad in PyTorch and complete the code below. 

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
############ Your code below ############

############       End       ############
train_losses, test_losses = train(model, optimizer)
populate_result(optim_results, 'AdaGrad', train_losses, test_losses)

### RMSprop

Complete the code below for RMSprop. Set smoothing constant to 0.9

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
############ Your code below ############

############       End       ############
train_losses, test_losses = train(model, optimizer)
populate_result(optim_results, 'RMSprop', train_losses, test_losses)

### Adam

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
############ Your code below ############

############       End       ############
train_losses, test_losses = train(model, optimizer)
populate_result(optim_results, 'Adam', train_losses, test_losses)

In [311]:
# Plot optimization methods training results
plot_loss(optim_results)

In [312]:
# Plot optimization methods validation results
plot_best_loss(optim_results)

## Regularization

https://pytorch.org/docs/stable/nn.html

In [307]:
# In order to see effects of regularization on validation set, we need to make some slight modification

## Smaller training set
trainset.train_data=trainset.train_data[:6000]
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

## Longer training epochs
train_epoches = 100
print_every = 500
reg_results = {}

### Baseline model

In [ ]:
torch.manual_seed(seed)
model = Baseline(input_size, n_feature, output_size).to(device)
optimizer = optim.SGD(model.parameters(),lr=0.01)
train_losses, test_losses = train(model, optimizer, num_epoch=train_epoches, print_every=print_every)
populate_result(reg_results, 'Baseline', train_losses, test_losses)

### L1/L2 regularization

L2 regularization is included in most optimizers in PyTorch and can be controlled with the __weight_decay__ parameter.
For L1 regularization, check out this post: https://discuss.pytorch.org/t/simple-l2-regularization/139

### Dropout Layer

1. How does Dropout introduce regularization effect?
2. Check out documentations for __nn.Dropout()__. Modify the Baseline model and add dropout layer to the fully connected layers.
3. Optionally, you may also check out documentations for __nn.Dropout2d()__ to learn how to add Dropout layer to convolution layers.

In [308]:
class DropoutNet(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(DropoutNet, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Sequential(nn.Conv2d(1, n_feature, kernel_size=5),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
        
        self.conv2 = nn.Sequential(nn.Conv2d(n_feature, n_feature, kernel_size=5),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
        self.fc = nn.Sequential(nn.Linear(n_feature*4*4, 50),
                                nn.ReLU(),
                                nn.Linear(50, 10))


    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc(x)
        return x

In [ ]:
torch.manual_seed(seed)
model = DropoutNet(input_size, n_feature, output_size, dropout_rate=0.5).to(device)
optimizer = optim.SGD(model.parameters(),lr=0.01)
train_losses, test_losses = train(model, optimizer, num_epoch=train_epoches, print_every=print_every)
populate_result(reg_results, 'Dropout', train_losses, test_losses)

### Batch Normalization
1. What's the advantage of using batch normalization?
2. Batch normalization also act as a form of regularization, why?
3. Modify the Baseline model and implement batch normalization in __BatchnormNet__. Think about where you may want to insert the batch normalization layer.

In [313]:
class BatchnormNet(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(BatchnormNet, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Sequential(nn.Conv2d(1, n_feature, kernel_size=5),
                                   nn.MaxPool2d(2),
                                   nn.ReLU())
        
        self.conv2 = nn.Sequential(nn.Conv2d(n_feature, n_feature, kernel_size=5),
                                   nn.MaxPool2d(2),
                                   nn.ReLU())
        self.fc = nn.Sequential(nn.Linear(n_feature*4*4, 50),
                                nn.ReLU(),
                                nn.Linear(50, 10))


    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc(x)
        return x

In [ ]:
torch.manual_seed(seed)
model = BatchnormNet(input_size, n_feature, output_size).to(device)
optimizer = optim.SGD(model.parameters(),lr=0.01)
train_losses, test_losses = train(model, optimizer, num_epoch=train_epoches, print_every=print_every)
populate_result(reg_results, 'Batchnorm', train_losses, test_losses)

In [ ]:
plot_loss(reg_results)

In [ ]:
plot_loss(reg_results, loss = 'val_loss')

## Exercise: parameter tunning

We've introduced multiple regularization and optimization techniques to improve your model. How can you combine these techniques and perform grid search to find out a set of parameters that maximize your model performance on validation set? Are there other model architectures you'd like to try?